In [ ]:
# ---
# title: "Test loading DLC SuperModel output"
# ---

# 🐭 Example notebook to test access to a DLC SuperModel .h5 output
# Students should be able to run this in Colab directly

import pandas as pd

# 🔗 URL to your GitHub file (replace <username>/<repo>)
url = "https://raw.githubusercontent.com/izbethMG-Teaching/pose2behav-book/main/assets/data/single-mouse_topview_10min_ 1_superanimal_topviewmouse_snapshot-fasterrcnn_mobilenet_v3_large_fpn-004_snapshot-hrnet_w32-004.h5"

# Try loading as a pandas DataFrame
df = pd.read_hdf(url)

print("✅ File loaded successfully!")
print("Shape:", df.shape)
print("First rows:")
display(df.head())

# --- Student exercise ---
# TODO: Write one line of code to print the list of columns

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](
https://colab.research.google.com/github/https://github.com/LizbethMG-Teaching/pose2behav-book/blob/main/notebooks/EDA.ipynb)